In [6]:
cd "C:\Users\drn455\Documents\GitHub\SQuID-Lab-quam"

C:\Users\drn455\Documents\GitHub\SQuID-Lab-quam


In [7]:
# cd C:\Users\jacob\OneDrive\Documents\GitHub\SQuID-Lab-quam

In [8]:
# cd C:\Users\MESO 1\Documents\GitHub\SQuID-Lab-quam

In [9]:
from squid_lab_quam.components.roots import SQuIDRoot1

machine = SQuIDRoot1()

2024-07-11 14:29:17,432 - qm - INFO     - Starting session: 53501d18-b9d2-4c41-9e87-374b0eefcb10


## Add information

In [10]:
from squid_lab_quam.components.information import Information

information_chalmers = Information(
    user_name="Jacob Hastrup",
    user_ku_tag="drn455",
    device_name="Chalmers/5q4c_v1_1",
    fridge_name='meso',
    # state_path=r"C:\Users\MESO 1\Documents\GitHub\SQuID-Lab-quam\squid_lab_quam\tests",
    # state_path=r"C:\Users\jacob\OneDrive\Documents\GitHub\SQuID-Lab-quam\squid_lab_quam\tests\state",
    state_path="C:/Users/drn455/Documents/GitHub/SQuID-Lab-quam/squid_lab_quam/tests/state",
)

information_soprano = Information(
    user_name="Jacob Hastrup",
    user_ku_tag="drn455",
    device_name="QuantWare/soprano_v3",
    fridge_name='T5',
    # state_path=r"C:\Users\MESO 1\Documents\GitHub\SQuID-Lab-quam\squid_lab_quam\tests",
    # state_path=r"C:\Users\jacob\OneDrive\Documents\GitHub\SQuID-Lab-quam\squid_lab_quam\tests\state",
    state_path="C:/Users/drn455/Documents/GitHub/SQuID-Lab-quam/squid_lab_quam/tests/state",
)

machine.information = information_soprano
machine.information.print_summary()
print(machine.information.device_subjectID)

information: Information
  user_name: "Jacob Hastrup"
  user_ku_tag: "drn455"
  device_name: "QuantWare/soprano_v3"
  fridge_name: "T5"
  state_path: "C:/Users/drn455/Documents/GitHub/SQuID-Lab-quam/squid_lab_quam/tests/state"
  data_path: "#./squid_lab_data_path"
B040001A01


## Add network

In [11]:
from squid_lab_quam.components.network import OPXNetwork, OctaveNetwork

network_meso = OPXNetwork(
    host="10.209.67.168",
    cluster_name="Cluster_1",
    octave_networks={
        "octave1": OctaveNetwork(
            octave_host="10.209.67.174", octave_port=80, controller="con1"
        )
    },
)

network_t5 = OPXNetwork(
        host= "10.209.96.1",
        cluster_name= "Cluster_1",
        octave_networks ={
            "octave1": OctaveNetwork(
                octave_host=   "10.209.96.11",
                octave_port= 80,
                controller= "con1"
            )
        }
)

machine.network = network_t5

## Add wiring

In [12]:
from squid_lab_quam.components.wiring import (
    OPXWiring,
    OPXFeedLineWiring,
    OPXIQChannelWiring,
)

wiring = OPXWiring(
    drive_lines={
        "q0": OPXIQChannelWiring(
            port_I=("con1", 3),
            port_Q=("con1", 4),
        ),
        "q1": OPXIQChannelWiring(
            port_I=("con1", 5),
            port_Q=("con1", 6),
        ),
        "q2": OPXIQChannelWiring(
            port_I=("con1", 7),
            port_Q=("con1", 8),
        ),
        "q3": OPXIQChannelWiring(
            port_I=("con1", 9),
            port_Q=("con1", 10),
        ),
        "q4": OPXIQChannelWiring(
            port_I=("con1", 9),
            port_Q=("con1", 10),
        ),
    },
    feed_lines={
        "feedline": OPXFeedLineWiring(
            output_I=("con1", 1),
            output_Q=("con1", 2),
            input_I=("con1", 1),
            input_Q=("con1", 2),
        )
    },
)

machine.wiring = wiring

## Add octaves

In [13]:
from squid_lab_quam.components.octave import OctaveSQuID

octaves = {
    "octave1": OctaveSQuID(
        ip=machine.network.octave_networks["octave1"].get_reference("octave_host"),
        port=machine.network.octave_networks["octave1"].get_reference("octave_port"),
    )
}
machine.octaves = octaves

## Add readout frequency converters

In [14]:
from squid_lab_quam.components.octave import (
    OctaveUpConverterSQuID,
    OctaveDownConverterSQuID,
)

readout_up_converter = OctaveUpConverterSQuID(LO_frequency=5e9)
octaves["octave1"].RF_outputs[machine.wiring.feed_lines['feedline'].default_octave_port_out()] = readout_up_converter

readout_down_converter = OctaveDownConverterSQuID(
    LO_frequency=readout_up_converter.get_reference("LO_frequency"),
)
octaves["octave1"].RF_inputs[1] = readout_down_converter

## Add qubits

In [15]:
from squid_lab_quam.components.qubits import ScQubit
from squid_lab_quam.components.resonators import ReadoutResonator
from quam.components.channels import IQChannel, InOutIQChannel
from squid_lab_quam.components.pulse_sets import PulseSetDragGaussian, PulseSetFlattopCosine
from quam.components.pulses import SquareReadoutPulse

qubit_names = ["q0", "q1", "q2", "q3"]

machine.shared_qubit_parameters = {
    "drag_gaussian_pulse_parameters": {"length": 40, "sigma": 7, "subtracted": True},
}


for idx, qubit_name in enumerate(qubit_names):
    # Add driving frequency converter
    drive_up_converter = OctaveUpConverterSQuID(LO_frequency=5.5e9)
    octaves["octave1"].RF_outputs[idx + 2] = drive_up_converter

    # Add qubit
    qubit = ScQubit(
        xy=IQChannel(
            opx_output_I=f"#/wiring/drive_lines/{qubit_name}/port_I",
            opx_output_Q=f"#/wiring/drive_lines/{qubit_name}/port_Q",
            frequency_converter_up=drive_up_converter.get_reference(),
        ),
        resonator=ReadoutResonator(
            channel=InOutIQChannel(
                opx_input_I=f"#/wiring/feed_lines/feedline/input_I",
                opx_input_Q=f"#/wiring/feed_lines/feedline/input_Q",
                opx_output_I=f"#/wiring/feed_lines/feedline/output_I",
                opx_output_Q=f"#/wiring/feed_lines/feedline/output_Q",
                frequency_converter_up=readout_up_converter.get_reference(),
                frequency_converter_down=readout_down_converter.get_reference(),
            )
        ),
        transition_frequencies=[6, 5],
    )
    qubit.pulse_sets = {
        "drag_gaussian": PulseSetDragGaussian(
            amplitude_180=0.5,
            amplitude_90=0.25,
            sigma="#/shared_qubit_parameters/drag_gaussian_pulse_parameters/sigma",
            length=machine.shared_qubit_parameters["drag_gaussian_pulse_parameters"].get_reference("length"),
            subtracted="#/shared_qubit_parameters/drag_gaussian_pulse_parameters/subtracted",
            anharmonicity="#../../anharmonicity",
        ),
        "flattop_cosine": PulseSetFlattopCosine(
            amplitude=0.1,
            rise_fall_time=16,
        )
    }
    
    machine.qubits[qubit_name] = qubit
    qubit.pulse_sets["drag_gaussian"].add_drive_pulses()
    qubit.pulse_sets["flattop_cosine"].add_drive_pulses()

    qubit.resonator.channel.operations["readout"] = SquareReadoutPulse(
        length=1000, amplitude=0.1, threshold=0.0, digital_marker="ON"
    )


In [17]:
machine

SQuIDRoot1(wiring=OPXWiring(drive_lines={'q0': OPXIQChannelWiring(port_I=('con1', 3), port_Q=('con1', 4)), 'q1': OPXIQChannelWiring(port_I=('con1', 5), port_Q=('con1', 6)), 'q2': OPXIQChannelWiring(port_I=('con1', 7), port_Q=('con1', 8)), 'q3': OPXIQChannelWiring(port_I=('con1', 9), port_Q=('con1', 10)), 'q4': OPXIQChannelWiring(port_I=('con1', 9), port_Q=('con1', 10))}, feed_lines={'feedline': OPXFeedLineWiring(output_I=('con1', 1), output_Q=('con1', 2), input_I=('con1', 1), input_Q=('con1', 2))}, flux_lines={}), network=OPXNetwork(host='10.209.96.1', cluster_name='Cluster_1', octave_networks={'octave1': OctaveNetwork(octave_host='10.209.96.11', octave_port=80, controller='con1')}), information=Information(user_name='Jacob Hastrup', user_ku_tag='drn455', device_name='QuantWare/soprano_v3', fridge_name='T5', state_path='C:/Users/drn455/Documents/GitHub/SQuID-Lab-quam/squid_lab_quam/tests/state', data_path='N:SCI-NBI-QDev\\SQuID Lab Data\\Devices\\QuantWare\\soprano_v3\\OPX Data'), qubi